# gu_SmartFigure: User Guide

This notebook is the primary, example-driven user guide for **gu_SmartFigure** — a small plotting helper for **SymPy expressions** in Jupyter, with a Plotly-first interactive widget.

**Audience:** mathematically-inclined researchers, students, and collaborators.

## Table of contents
- [What this package is for](#what-this-package-is-for-and-what-it-is-not)
- [Installation & Requirements](#installation--requirements)
- [Quickstart (5 minutes)](#quickstart-5-minutes)
- [Core Concepts](#core-concepts)
- [Common Tasks / Recipes](#common-tasks--recipes)
- [Applications](#applications-24-mini-projects)
- [Advanced Guide](#advanced-guide)
- [API Reference](#api-reference-lightweight)
- [Next Steps](#next-steps)


## What this package is for (and what it is not)

### Purpose
`gu_SmartFigure` is designed for *small-to-medium* interactive exploration of **1D real-valued functions** defined symbolically in SymPy.

Typical use cases:
- Quickly sketch/inspect expressions while teaching or doing calculations.
- Explore *parameterized* families of functions with sliders.
- Keep workflows reproducible: the “state” is the SymPy expression + numeric parameter values.

### Non-goals
- Not a general plotting library (no 3D, no scatter/markers, no layout engine).
- Not a replacement for a full CAS or numeric PDE/ODE framework.
- Not intended for huge datasets; it evaluates on a 1D grid and updates traces.

### Mental model (Model / Controller / View)
- **Model:** `Plot` — holds a SymPy expression, sampling settings, and style.
- **Controller:** `SmartFigure` — owns plots, a viewport (`x_range`, `y_range`), and recomputation logic.
- **View:** `PlotBackend` — rendering seam (Plotly `FigureWidget` by default; Matplotlib optional).


## Installation & Requirements

### Minimal requirements
- Python 3.9+
- `numpy`
- `sympy`

### Optional dependencies
`gu_SmartFigure` intentionally *lazy-imports* heavy UI dependencies. Features appear when optional packages are present.

| Optional package | Install hint (generic) | Enables |
|---|---|---|
| `ipywidgets` | `pip install ipywidgets` | Jupyter widget container + sliders |
| `plotly` (with `FigureWidget`) | `pip install plotly` | Interactive Plotly-backed widget |
| `matplotlib` | `pip install matplotlib` | Script-friendly Matplotlib backend (`MplBackend`) |

### Environment notes
- **Google Colab is not supported** for the interactive widget.
- **JupyterLite/Pyodide** is supported *if* your build bundles the widget manager + Plotly.


In [ ]:
# Notebook setup: imports + environment detection
from __future__ import annotations

import importlib
import sys
from typing import Any, Dict, Tuple

import numpy as np
import sympy as sp

np.set_printoptions(precision=4, suppress=True)

def _try_import(name: str):
    try:
        return importlib.import_module(name), None
    except Exception as e:
        return None, e

def import_gu_smartfigure():
    """Import the package from likely module paths.

    If your project installs it under a different name, add that name here.
    """
    candidates = [
        "gu_SmartFigure",                        # primary name used in the module docstring
        "gu_toolkit.plugins.SmartFigure",        # common plugin-style namespace
        "gu_toolkit.plugins.SmartFigure.SmartFigure",
    ]
    last_err = None
    for modname in candidates:
        m, err = _try_import(modname)
        if m is not None:
            # Ensure subsequent `from gu_SmartFigure import ...` works even if imported via an alias.
            sys.modules.setdefault("gu_SmartFigure", m)
            return m
        last_err = err
    raise ImportError(
        "Could not import gu_SmartFigure from any known module path.\n"
        "Tried: " + ", ".join(candidates) + "\n"
        f"Last error: {last_err}"
    )

pkg = import_gu_smartfigure()

# Discover the public entrypoints
PUBLIC = {name: getattr(pkg, name) for name in getattr(pkg, "__all__", [])}
list(PUBLIC.keys())


The cell above imports the package and introspects its public entrypoints via `__all__`.

In the rest of this notebook we use the public API directly.

In [ ]:
# Public entrypoints (imported from the package)
from gu_SmartFigure import (  # type: ignore
    VIEWPORT,
    SmartSlider,
    Style,
    PlotBackend,
    MplBackend,
    PlotlyTraceHandle,
    PlotlyBackend,
    Plot,
    SmartFigure,
)

list(PUBLIC.keys())


## Quickstart (5 minutes)

We’ll do a minimal, reproducible workflow:
1) define a SymPy expression with a parameter,
2) create a `SmartFigure` without displaying immediately,
3) add a plot,
4) change the parameter value,
5) produce a concrete output (data arrays, and optionally an interactive widget).


In [ ]:
# Quickstart: define a parameterized family f(x) = exp(-a x^2)
x = sp.Symbol("x")
a = sp.Symbol("a")

expr = sp.exp(-a * x**2)

sample_count = 400
fig = SmartFigure(var=x, x_range=(-3, 3), y_range=(-0.1, 1.1), samples=sample_count, show_now=False)

p = fig.plot(expr, name="gaussian")  # symbol inferred: x; parameter inferred: a

# Parameter values live in the figure's registry.
reg = fig.parameter_registry
param_a = reg.get_param(a)

# Set a deterministic value
param_a.value = 1.0

# Concrete output that always works: sample (x, y) arrays
x_arr, y_arr = p.compute_data(fig.x_range, global_samples=sample_count)

assert x_arr.shape == y_arr.shape
x_arr[:5], y_arr[:5]


In [ ]:
# Optional: show an interactive widget (requires ipywidgets + plotly FigureWidget)
import importlib

has_widgets = importlib.util.find_spec("ipywidgets") is not None
has_plotly = importlib.util.find_spec("plotly") is not None

def has_figurewidget() -> bool:
    if not has_plotly:
        return False
    try:
        import plotly.graph_objects as go
        _ = go.FigureWidget  # attribute exists
        # Instantiation checks whether ipywidgets is installed & compatible
        go.FigureWidget()
        return True
    except Exception:
        return False

if has_widgets and has_figurewidget():
    # Separate construction from display side-effect.
    widget = fig.widget
    from IPython.display import display
    display(widget)
else:
    print("Interactive widget not available here (needs ipywidgets + plotly FigureWidget).")
    print("You can still: (1) sample arrays via Plot.compute_data, or (2) use MplBackend below.")


### Optional UI primitives: `SmartSlider` and `PlotlyBackend`

If you are building custom UIs, `SmartSlider` is a small helper that returns an `ipywidgets` slider, and `PlotlyBackend` provides a minimal backend implementation.

This cell demonstrates **graceful failure** when widgets aren’t installed.


In [ ]:
def demo_optional_ui():
    # SmartSlider is only meaningful when ipywidgets is available.
    s = SmartSlider("a", min=0.0, max=2.0, step=0.1, value=1.0, readout_format=".2f")
    return s

try:
    s = demo_optional_ui()
    print("SmartSlider created:", type(s))
except Exception as e:
    print("SmartSlider unavailable:", type(e).__name__, "-", e)

# PlotlyBackend may also fail if FigureWidget can't be created.
try:
    pb = PlotlyBackend()
    print("PlotlyBackend created:", type(pb))
except Exception as e:
    print("PlotlyBackend unavailable:", type(e).__name__, "-", e)


### A script-friendly plot (Matplotlib backend)

Even if you do **not** have widgets available, you can still visualize the sampled arrays with `MplBackend`.


In [ ]:
# Matplotlib fallback (no widgets required)
mpl = MplBackend(figsize=(6, 3), dpi=110)
mpl.set_viewport(fig.x_range, fig.y_range)

_ = mpl.add_plot("gaussian", x_arr, y_arr, Style(color="black", width=2.0))
mpl.request_redraw()

# Programmatic verification: the peak should be at x=0 with value ~ 1
idx0 = int(np.argmin(np.abs(x_arr)))
assert abs(x_arr[idx0]) < 1e-12
assert abs(y_arr[idx0] - 1.0) < 1e-12


## Core Concepts

This section introduces the key objects you’ll interact with most.


### 1) `SmartFigure` is the controller

`SmartFigure` owns:
- a viewport (`x_range`, `y_range`),
- a global sampling resolution (`samples`),
- a plot registry keyed by plot name,
- a parameter registry (`parameter_registry`) for scalar parameters.

A figure can be used *headlessly* (sample arrays) or with an interactive widget (sliders + Plotly view).

**Gotchas**
- The interactive widget is intentionally *not* supported in Google Colab.
- The Plotly backend uses `FigureWidget` which requires `ipywidgets`.


In [ ]:
# SmartFigure basics: viewport and global sampling
x = sp.Symbol("x")
fig = SmartFigure(var=x, x_range=(-2, 2), y_range=(-2, 2), samples=200, show_now=False)

assert fig.x_range == (-2.0, 2.0)
assert fig.y_range == (-2.0, 2.0)

# Updating ranges triggers resampling on next draw (or immediately if a backend is live).
fig.x_range = (-4, 4)
fig.x_range


### 2) Expression analysis: independent variable vs parameters

When you call `fig.plot(expr, ...)`, the figure decides:
- the **independent symbol** (the x-axis variable), and
- which free symbols become **parameters**.

Rules (informal):
- If the expression has **0 free symbols**, it’s a constant function.
- If it has **1 free symbol**, that symbol is the independent variable *unless* you explicitly pass a different `symbol`.
- If it has **2+ free symbols**, you must specify `symbol=...`; all other symbols become parameters.

This keeps parameter ordering deterministic and slider-friendly.

**Gotchas**
- Multi-symbol expressions require `symbol=...`.
- “Parameterized constant”: if `expr` depends only on `a` but you set `symbol=x`, you get a horizontal line controlled by `a`.


In [ ]:
# Expression analysis examples
x = sp.Symbol("x")
a = sp.Symbol("a")
b = sp.Symbol("b")

fig = SmartFigure(var=x, show_now=False, samples=50)

# (i) Constant expression: no free symbols
p_const = fig.plot(5, name="const")  # independent symbol defaults to fig.var
assert p_const.param_symbols == tuple()

# (ii) One symbol: inferred independent variable
p_sin = fig.plot(sp.sin(x), name="sinx")
assert p_sin.symbol == x and p_sin.param_symbols == tuple()

# (iii) Parameterized constant: expr has symbol 'a', but we want to plot vs x
p_param_const = fig.plot(a, name="a_as_line", symbol=x)
assert p_param_const.symbol == x and p_param_const.param_symbols == (a,)

# (iv) Two symbols: must specify independent variable explicitly
p_two = fig.plot(sp.sin(a * x) + b, name="two_params", symbol=x)
assert p_two.param_symbols == (a, b)

[p_const, p_sin, p_param_const, p_two]


### 3) Sampling: viewport, `domain`, and `VIEWPORT`

Each `Plot` can sample on either:
- the figure viewport (use `domain=VIEWPORT`), or
- a plot-specific domain `(xmin, xmax)`.

Similarly, each plot can inherit the figure's sample count with `samples=VIEWPORT`.

**Gotchas**
- If a plot’s domain does not intersect the current viewport, it samples nothing (empty arrays).
- `domain` values are interpreted as floats; `None` can be used for “unbounded”, but unbounded domains will be clipped by the viewport.


In [ ]:
# Domain and sampling examples
x = sp.Symbol("x")
fig = SmartFigure(var=x, x_range=(-2, 2), samples=200, show_now=False)

p_view = fig.plot(sp.sin(x), name="view", domain=VIEWPORT, samples=VIEWPORT)
p_local = fig.plot(sp.sin(x), name="local", domain=(-1, 1), samples=50)

x1, y1 = p_view.compute_data(fig.x_range, global_samples=200)
x2, y2 = p_local.compute_data(fig.x_range, global_samples=200)

assert len(x1) == 200
assert len(x2) == 50
assert x2.min() >= -1 and x2.max() <= 1
(len(x1), len(x2), (x2.min(), x2.max()))


### 4) Styling: `Style` and reactive updates

`Style` is a small validated container:
- `visible: bool`
- `color: str | None`
- `width: float | None` (must be > 0)
- `opacity: float | None` (0..1)
- `linestyle: {'-', '--', ':', '-.'} | None`

You can pass style when creating a plot, or mutate `plot.style` later.

**Gotchas**
- Invalid style values raise a `ValueError`/`TypeError` early.
- In a live widget, `plot.style.<field> = ...` triggers a redraw.


In [ ]:
# Style validation & usage
st = Style(color="red", width=2.5, opacity=0.8, linestyle="--")
st


In [ ]:
# Aliases are accepted (lw/linewidth, alpha, ls, vis)
st2 = Style(lw=1.5, alpha=0.5, ls=":", vis=True)
st2


In [ ]:
# Invalid style examples (expected to raise)
def show_error(fn):
    try:
        fn()
    except Exception as e:
        print(type(e).__name__ + ":", e)

show_error(lambda: Style(width=-1))
show_error(lambda: Style(opacity=2))
show_error(lambda: Style(linestyle="..."))


### 5) Parameters: registry values and recompute orchestration

Parameters are stored in `fig.parameter_registry` as scalar `SmartParameter` objects.

At a user level:
- Create/update plots that involve parameters.
- Read or set parameter values via the registry.
- If a live backend exists, changing a parameter triggers recomputation of only the dependent plots.

**Gotchas**
- Dependency-tracking recomputation happens only when a backend is live. If you are sampling arrays manually, you recompute by calling `plot.compute_data(...)`.


In [ ]:
# Parameter registry: set values and observe effect on sampled arrays
x = sp.Symbol("x")
a = sp.Symbol("a")

fig = SmartFigure(var=x, x_range=(-3, 3), samples=200, show_now=False)
p = fig.plot(sp.exp(-a * x**2), name="gauss")

reg = fig.parameter_registry
reg.get_param(a).value = 0.5
x0, y0 = p.compute_data(fig.x_range, global_samples=200)

reg.get_param(a).value = 2.0
x1, y1 = p.compute_data(fig.x_range, global_samples=200)

# Verification: larger a -> narrower Gaussian -> smaller value at x=1
i = int(np.argmin(np.abs(x0 - 1.0)))
assert y1[i] < y0[i]
(y0[i], y1[i])


## Common Tasks / Recipes

Short, searchable recipe cards.


### Recipe: Add a plot with an explicit name

**Task:** plot `sin(x)` with a custom style.


In [ ]:
x = sp.Symbol("x")
fig = SmartFigure(var=x, show_now=False)
p = fig.plot(sp.sin(x), name="sine", style={"color": "green", "width": 2.0})

# Verify
assert p.name == "sine"
assert p.param_symbols == tuple()
p


### Recipe: Update an existing plot (same name)

Calling `fig.plot(..., name=<existing>)` updates the existing plot.


In [ ]:
x = sp.Symbol("x")
fig = SmartFigure(var=x, show_now=False)
p = fig.plot(sp.sin(x), name="f")
p2 = fig.plot(sp.cos(x), name="f")  # update in place

assert p is p2
assert sp.simplify(p2.expr - sp.cos(x)) == 0
p2


### Recipe: Hide/show a plot


In [ ]:
x = sp.Symbol("x")
fig = SmartFigure(var=x, show_now=False)
p = fig.plot(sp.sin(x), name="f")

p.visible = False
assert p.visible is False
p.visible = True
assert p.visible is True
p


### Recipe: Remove a plot / clear the figure


In [ ]:
x = sp.Symbol("x")
fig = SmartFigure(var=x, show_now=False)
fig.plot(sp.sin(x), name="a")
fig.plot(sp.cos(x), name="b")

fig.remove("a")

# Verify: removing again should fail
try:
    fig.remove("a")
except Exception as e:
    print(type(e).__name__ + ":", e)

fig.clear()

# Verify: clearing again is a no-op
fig.clear()


### Recipe: Use LaTeX/MathJax in plot labels (Plotly)

Plotly can render inline MathJax when the name contains `$...$`.

Notes:
- Use `\$` for a literal dollar sign.
- An unmatched `$` will raise a `GuideError` (to prevent silent client-side failures).


In [ ]:
# This cell does not require the widget; it just illustrates the naming rule.
# If PlotlyBackend is available, these names appear in the legend.
good = r"$\sin(x)$ and cost \$5"
bad = r"$\sin(x)$ and then $oops"  # unmatched

print("Good label:", good)
print("Bad label :", bad)


### Recipe: Export sampled data

A lightweight way to interoperate with other tools is to sample arrays and export.


In [ ]:
import csv
from pathlib import Path

x = sp.Symbol("x")
fig = SmartFigure(var=x, x_range=(-2, 2), samples=100, show_now=False)
p = fig.plot(sp.sin(x), name="sine")

x_arr, y_arr = p.compute_data(fig.x_range, global_samples=100)

out = Path("sine_samples.csv")
with out.open("w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["x", "y"])
    w.writerows(zip(x_arr.tolist(), y_arr.tolist()))

out, out.stat().st_size


## Applications (2–4 mini-projects)

Each mini-project is deterministic and intended to be understandable to advanced undergraduates / beginning grad students.


### Application 1: Parameter study — Gaussian width vs. parameter

**Motivating question:** How does the width of a Gaussian `exp(-a x^2)` change as `a` varies?

Workflow:
1) Build `f_a(x) = exp(-a x^2)`.
2) Sample on a fixed domain.
3) For a small grid of `a` values, compute the *full width at half maximum* (FWHM).

We avoid any randomness and keep the dataset small.


In [ ]:
x = sp.Symbol("x")
a = sp.Symbol("a")

sample_count = 2000
fig = SmartFigure(var=x, x_range=(-4, 4), samples=sample_count, show_now=False)
p = fig.plot(sp.exp(-a * x**2), name="gaussian")

def fwhm_from_samples(xv: np.ndarray, yv: np.ndarray) -> float:
    """Compute FWHM from samples of a unimodal symmetric peak at x=0."""
    if len(xv) == 0:
        return float("nan")
    ymax = float(np.max(yv))
    if ymax <= 0:
        return float("nan")
    half = 0.5 * ymax
    mid = int(np.argmin(np.abs(xv)))  # x≈0
    right = np.where(yv[mid:] <= half)[0]
    if len(right) == 0:
        return float("nan")
    j = mid + int(right[0])
    return float(2 * abs(xv[j]))

a_values = [0.25, 0.5, 1.0, 2.0, 4.0]
rows = []
for aval in a_values:
    fig.parameter_registry.get_param(a).value = aval
    xv, yv = p.compute_data(fig.x_range, global_samples=sample_count)
    rows.append((aval, fwhm_from_samples(xv, yv)))

rows


In [ ]:
# A quick visualization (Matplotlib backend)
mpl = MplBackend(figsize=(6, 3), dpi=110)
mpl.set_viewport(fig.x_range, (-0.1, 1.1))

# Plot three representative curves
for aval in (0.5, 1.0, 2.0):
    fig.parameter_registry.get_param(a).value = aval
    xv, yv = p.compute_data(fig.x_range, global_samples=sample_count)
    mpl.add_plot(f"a={aval}", xv, yv, Style(width=2.0))

mpl.request_redraw()

# Verify monotonic trend: FWHM decreases with a
fwhms = [w for _, w in rows]
assert all(fwhms[i] > fwhms[i+1] for i in range(len(fwhms)-1))


**What we learned:** increasing `a` makes the Gaussian narrower.

**How to adapt:** replace `exp(-a x^2)` by another parameterized family and compute a diagnostic (peak location, oscillation count, etc.).


### Application 2: Error curves — Taylor approximation of `sin(x)`

**Motivating question:** How do Taylor polynomials approximate `sin(x)` on `[-π, π]`?

Workflow:
1) Generate Taylor polynomials of odd order.
2) Sample both functions.
3) Plot the absolute error.


In [ ]:
x = sp.Symbol("x")
f = sp.sin(x)

def taylor_sin(order: int) -> sp.Expr:
    # SymPy series returns an O(x**n) term; remove it.
    return sp.series(sp.sin(x), x, 0, order+1).removeO()

orders = [1, 3, 5, 7, 9]
polys = {n: taylor_sin(n) for n in orders}
polys[5]


In [ ]:
sample_count = 1200
fig = SmartFigure(var=x, x_range=(-float(np.pi), float(np.pi)), samples=sample_count, show_now=False)

p_true = fig.plot(f, name="sin(x)", style={"width": 2.0})

# Create polynomial plots (no parameters)
poly_plots = []
for n in orders:
    poly_plots.append(fig.plot(polys[n], name=f"taylor_{n}", style={"opacity": 0.9}))

# Compute and plot absolute error for a single order as a concrete output
n = 7
xv, y_true = p_true.compute_data(fig.x_range, global_samples=sample_count)
xv, y_approx = poly_plots[orders.index(n)].compute_data(fig.x_range, global_samples=sample_count)

err = np.abs(y_true - y_approx)
assert err.shape == xv.shape

# Visualize error with Matplotlib backend
mpl = MplBackend(figsize=(6, 3), dpi=110)
mpl.set_viewport(fig.x_range, (0, float(np.max(err))*1.05))
mpl.add_plot(f"|sin - taylor_{n}|", xv, err, Style(color="red", width=2.0))
mpl.request_redraw()

float(np.max(err))


**What we learned:** on a fixed interval, higher-order Taylor polynomials reduce the error near 0, but can still grow toward the endpoints.

**How to adapt:** compare other approximations (Padé, Chebyshev) by plugging in different SymPy expressions.


### Application 3: A small “family of curves” mini-project — damped oscillations

**Motivating question:** How does a damping parameter change oscillations?

We consider
\[
f_{a}(x) = e^{-a x^2}\,\cos(6x).
\]

We’ll:
1) plot the curve,
2) compute a simple diagnostic (RMS amplitude on the interval),
3) show how the diagnostic changes with `a`.


In [ ]:
x = sp.Symbol("x")
a = sp.Symbol("a")

expr = sp.exp(-a * x**2) * sp.cos(6*x)

sample_count = 2000
fig = SmartFigure(var=x, x_range=(-4, 4), samples=sample_count, show_now=False)
p = fig.plot(expr, name="damped_cos")

def rms(y: np.ndarray) -> float:
    return float(np.sqrt(np.mean(np.square(y))))

a_values = np.linspace(0.0, 1.0, 6)
rms_values = []

for aval in a_values:
    fig.parameter_registry.get_param(a).value = float(aval)
    xv, yv = p.compute_data(fig.x_range, global_samples=sample_count)
    rms_values.append(rms(yv))

list(zip(a_values.tolist(), rms_values))


In [ ]:
# Quick plot of RMS vs parameter a
mpl = MplBackend(figsize=(5, 3), dpi=110)
mpl.set_viewport((float(a_values.min()), float(a_values.max())), (0, max(rms_values)*1.05))
mpl.add_plot("RMS amplitude", a_values, np.array(rms_values), Style(width=2.0))
mpl.request_redraw()

# Verify monotonic decrease (for this family / interval)
assert all(rms_values[i] >= rms_values[i+1] for i in range(len(rms_values)-1))


**What we learned:** damping reduces overall amplitude.

**How to adapt:** replace `cos(6x)` by another oscillatory term or change the damping envelope.


## Advanced Guide

This section is for power users and collaborators extending the package.


### Customization & configuration

At the `SmartFigure` level:
- `x_range`, `y_range`: viewport (floats)
- `samples`: global sampling count
- `parameter_registry`: share parameters across figures or isolate them

At the `Plot` level:
- `domain`: `(xmin, xmax)` or `VIEWPORT`
- `samples`: integer or `VIEWPORT`
- `style`: `Style(...)` or a mapping passed via `fig.plot(..., style=...)`

Tip: choose sample counts based on the highest frequency you expect to resolve; avoid excessive points by default.


In [ ]:
# Share one parameter registry across multiple figures
x = sp.Symbol("x")
a = sp.Symbol("a")

fig1 = SmartFigure(var=x, show_now=False)
shared = fig1.parameter_registry  # public attribute
fig2 = SmartFigure(var=x, parameter_registry=shared, show_now=False)

p1 = fig1.plot(sp.exp(-a*x**2), name="gauss1")
p2 = fig2.plot(sp.cos(a*x), name="cos2")

shared.get_param(a).value = 2.0

# Both figures read the same registry value when sampling
_, y1 = p1.compute_data(fig1.x_range, global_samples=200)
_, y2 = p2.compute_data(fig2.x_range, global_samples=200)

float(y1[int(len(y1)/2)]), float(y2[int(len(y2)/2)])


### Extending the package

#### 1) Implementing a custom backend (`PlotBackend`)

If you want to render somewhere else (another widget system, a file export, etc.), implement the `PlotBackend` protocol:
- `add_plot(name, x, y, style) -> handle`
- `update_plot(handle, x, y)`
- `apply_style(handle, style)`
- `remove_plot(handle)`
- `set_viewport(x_range, y_range)`
- `request_redraw()`

**Invariants your backend must preserve:**
- `x` and `y` are 1D NumPy arrays of the same length.
- Parameter symbols are scalars; if the underlying expression is constant in `x`, the model broadcasts it to the `x` grid.

Below is a tiny backend that records updates (useful for testing).


In [ ]:
# Example backend: record calls for tests (no UI)
from typing import List, Optional

class RecordingBackend:
    def __init__(self):
        self.calls: List[Tuple[str, Any]] = []

    def add_plot(self, name: str, x: np.ndarray, y: np.ndarray, style: Style) -> str:
        h = f"handle:{name}"
        self.calls.append(("add_plot", name, x.shape, y.shape, repr(style)))
        return h

    def update_plot(self, handle: str, x: np.ndarray, y: np.ndarray) -> None:
        self.calls.append(("update_plot", handle, x.shape, y.shape))

    def apply_style(self, handle: str, style: Style) -> None:
        self.calls.append(("apply_style", handle, repr(style)))

    def remove_plot(self, handle: str) -> None:
        self.calls.append(("remove_plot", handle))

    def set_viewport(self, x_range: Tuple[float, float], y_range: Optional[Tuple[float, float]]) -> None:
        self.calls.append(("set_viewport", x_range, y_range))

    def request_redraw(self) -> None:
        self.calls.append(("request_redraw",))

rb = RecordingBackend()
h = rb.add_plot("f", np.array([0.0, 1.0]), np.array([1.0, 2.0]), Style())
rb.update_plot(h, np.array([0.0]), np.array([0.0]))
rb.calls[:3]


#### 1b) `PlotlyTraceHandle` (for backend implementers)

`PlotlyTraceHandle` is a small dataclass used by `PlotlyBackend` to refer to traces stably (via Plotly's `uid`). Most users never need this directly, but it is useful when implementing tooling around the Plotly backend.


In [ ]:
from dataclasses import asdict

h = PlotlyTraceHandle(uid="example-uid-123")
asdict(h)


#### 2) Custom SymPy functions with vectorized numerical implementations (`NamedFunction`)

Sometimes you want to plot an *opaque* function that SymPy can represent symbolically, but where you want a dedicated vectorized numeric implementation.

The `NamedFunction` decorator (additional input) is intended to convert either:
- a Python function **or**
- a class with `symbolic(...)` + `numeric(...)`

into a dynamic SymPy `Function` class.

How this usually connects to `gu_SmartFigure`:
- Your numeric implementation is used by the package’s compiler (often via a custom mapping in `numpify`).
- The symbolic definition is used for algebraic manipulation and printing.

Below is a **minimal** implementation that demonstrates the *shape* of the API. If your project already provides `NamedFunction`, prefer importing that.


In [ ]:
# Minimal NamedFunction implementation (works for SymPy + lambdify; integration with numpify is package-specific)
import inspect
from typing import Callable, Type, Union

def NamedFunction(obj: Union[Callable, Type]) -> Type[sp.Function]:
    """Convert a Python callable or a provider class into a SymPy Function subclass.

    - If `obj` is a function: we use it as a symbolic rule via `eval`.
    - If `obj` is a class: we instantiate it and use:
        * `.symbolic(*args)` to optionally rewrite/simplify symbolically
        * `.numeric(*args)` as a vectorized numerical implementation (for lambdify maps)
    """
    if inspect.isclass(obj):
        provider = obj()

        class _F(sp.Function):  # type: ignore[misc]
            @classmethod
            def eval(cls, *args):
                try:
                    out = provider.symbolic(*args)
                except Exception:
                    return None
                return None if out is None else sp.sympify(out)

        _F.__name__ = obj.__name__
        setattr(_F, "_numeric_impl", provider.numeric)
        return _F

    if callable(obj):
        func = obj

        class _F(sp.Function):  # type: ignore[misc]
            @classmethod
            def eval(cls, *args):
                try:
                    return sp.sympify(func(*args))
                except Exception:
                    return None

        _F.__name__ = getattr(func, "__name__", "NamedFunction")
        return _F

    raise TypeError("NamedFunction expects a callable or a class.")

# Example: Sinc(x) = sin(x)/x with a safe numeric implementation
@NamedFunction
class Sinc:
    def symbolic(self, x):
        # Optional symbolic rewrite
        return sp.Piecewise((1, sp.Eq(x, 0)), (sp.sin(x)/x, True))

    def numeric(self, x):
        x = np.asarray(x, dtype=float)
        out = np.ones_like(x)
        m = x != 0
        out[m] = np.sin(x[m]) / x[m]
        return out

# Use in expressions
x = sp.Symbol("x")
expr = Sinc(x) * sp.cos(x)

# Numerical evaluation via lambdify with a custom mapping:
f_num = sp.lambdify(x, expr, modules=[{Sinc: getattr(Sinc, "_numeric_impl")}, "numpy"])
xs = np.linspace(-5, 5, 11)
ys = f_num(xs)

assert ys.shape == xs.shape
ys[:5]


If your `gu_SmartFigure` compiler (`numpify`) supports custom function mappings, register `Sinc`’s numeric implementation there. Once registered, you can plot expressions containing `Sinc(x)` normally.


### Performance & scaling notes

Pragmatic guidance:
- **Sampling density** dominates cost: start with 200–1000 points, increase only when necessary.
- Compiling expressions can be expensive; reuse plots and update parameter values instead of recreating plots.
- For very expensive expressions, consider:
  - simplifying the SymPy expression first,
  - providing a custom numeric implementation (see `NamedFunction`),
  - reducing the viewport or sample count.

High-level profiling idea:
- Time `plot.compute_data(...)` in isolation for representative parameter values.


In [ ]:
# Micro-timing example (deterministic)
import time

x = sp.Symbol("x")
a = sp.Symbol("a")
expr = sp.exp(-a*x**2) * sp.cos(10*x)

sample_count = 2000
fig = SmartFigure(var=x, x_range=(-5, 5), samples=sample_count, show_now=False)
p = fig.plot(expr, name="f")

fig.parameter_registry.get_param(a).value = 1.0

t0 = time.perf_counter()
_ = p.compute_data(fig.x_range, global_samples=sample_count)
t1 = time.perf_counter()

print(f"compute_data: {(t1-t0)*1000:.1f} ms for {sample_count} samples")


### Interoperability

Common patterns:
- Use `Plot.compute_data(...)` to get NumPy arrays for SciPy/numerical routines.
- Export arrays to CSV for other software.
- Use `MplBackend` for static figures in scripts or PDF generation.

`gu_SmartFigure` intentionally keeps the boundary simple: `x`/`y` arrays in, `x`/`y` arrays out.


### Troubleshooting

Common issues and fixes:

- **`GuideError: ipywidgets is required...`**
  - Install `ipywidgets` and (in JupyterLab) enable the widget extension as appropriate for your environment.

- **`GuideError: Plotly (with FigureWidget support) is required...`**
  - Install `plotly`. In some environments you may also need `ipywidgets`.

- **Multi-symbol expression error**
  - If your expression has >1 free symbol, pass `symbol=...` explicitly.

- **Unmatched `$` in plot name**
  - Use `$...$` for LaTeX segments; use `\$` for a literal dollar.

- **Empty arrays returned**
  - Your plot `domain` may not intersect the current `x_range` viewport.

When reporting a bug, produce a minimal reproducible example:
1) the SymPy expression(s),
2) `x_range`, `samples`,
3) parameter values.


## API Reference (lightweight)

This is a curated index of the public API. For authoritative parameter details, consult docstrings (`help(...)`).

### Top-level
- `VIEWPORT`: sentinel meaning “inherit viewport/sampling from the figure”.
- `SmartSlider`: widget factory (only relevant when building the interactive UI).

### Configuration
- `Style`: validated style container.

### Backends
- `PlotBackend`: protocol defining the backend contract.
- `MplBackend`: Matplotlib implementation of `PlotBackend`.
- `PlotlyBackend`: Plotly `FigureWidget` implementation.
- `PlotlyTraceHandle`: stable identifier for Plotly traces.

### Core
- `Plot`: one plotted expression; key methods:
  - `compute_data(viewport_range, global_samples, param_values=None)`
  - `update(...)`
- `SmartFigure`: figure/controller; key methods/properties:
  - `plot(expr, name=..., symbol=..., domain=..., samples=..., style=...)`
  - `remove(name)`, `clear()`
  - `x_range`, `y_range`
  - `parameter_registry`
  - `widget` and `show()` (when UI deps are available)


In [ ]:
# Jump to docstrings quickly
import inspect

print("SmartFigure.plot signature:")
print(inspect.signature(SmartFigure.plot))

print("\nPlot.compute_data signature:")
print(inspect.signature(Plot.compute_data))


## Next Steps

Suggested reading order:
1) Quickstart
2) Core Concepts (especially expression analysis + parameters)
3) Common Tasks / Recipes
4) Applications
5) Advanced Guide

Validation / tests:
- In the source tree, look for a small deterministic test suite (often under `tests/`).
- A good smoke test avoids importing optional UI dependencies and exercises `Plot.compute_data` + expression analysis.

If your project ships a separate **TestSuite notebook**, run it after installation to validate your environment.
